In [25]:
from langchain_google_cloud_sql_pg import (
    PostgresEngine,
    PostgresSaver,
)
from src.config import env
from langchain_core.runnables import RunnableConfig
from langchain.load import dumpd

In [3]:
async def get_ckeckpoint():
    engine = await PostgresEngine.afrom_instance(
        project_id=env.PROJECT_ID,
        region=env.LOCATION,
        instance=env.INSTANCE,
        database=env.DATABASE,
        user=env.DATABASE_USER,
        password=env.DATABASE_PASSWORD,
        engine_args={"pool_pre_ping": True, "pool_recycle": 300},
    )
    checkpointer = await PostgresSaver.create(engine=engine)
    return checkpointer

In [ ]:
checkpointer = await get_ckeckpoint()

In [38]:
user_id = "1154af1a-7bf6-441d-ae06-4590a66c0d3d"
config = RunnableConfig(configurable={"thread_id": user_id})
l = await checkpointer.aget(config=config)

In [40]:
user_id = "user_123"
config = RunnableConfig(configurable={"thread_id": user_id})
m = await checkpointer.aget(config=config)

In [ ]:
from enum import Enum
from typing import Sequence
from langchain_core.messages import AIMessage, BaseMessage, ToolMessage


class ErrorCode(Enum):
    GRAPH_RECURSION_LIMIT = "GRAPH_RECURSION_LIMIT"
    INVALID_CONCURRENT_GRAPH_UPDATE = "INVALID_CONCURRENT_GRAPH_UPDATE"
    INVALID_GRAPH_NODE_RETURN_VALUE = "INVALID_GRAPH_NODE_RETURN_VALUE"
    MULTIPLE_SUBGRAPHS = "MULTIPLE_SUBGRAPHS"
    INVALID_CHAT_HISTORY = "INVALID_CHAT_HISTORY"


def create_error_message(*, message: str, error_code: ErrorCode) -> str:
    return (
        f"{message}\n"
        "For troubleshooting, visit: https://python.langchain.com/docs/"
        f"troubleshooting/errors/{error_code.value}"
    )


def _validate_chat_history(
    messages: Sequence[BaseMessage],
):
    """Validate that all tool calls in AIMessages have a corresponding ToolMessage."""
    all_tool_calls = [
        tool_call
        for message in messages
        if isinstance(message, AIMessage)
        for tool_call in message.tool_calls
    ]
    tool_call_ids_with_results = {
        message.tool_call_id for message in messages if isinstance(message, ToolMessage)
    }
    tool_calls_without_results = [
        tool_call
        for tool_call in all_tool_calls
        if tool_call["id"] not in tool_call_ids_with_results
    ]

    return all_tool_calls

In [ ]:
messages_l = l["channel_values"]["messages"]


ll = _validate_chat_history(messages_l)

isinstance(ll, ToolMessage)

False

In [ ]:
messages_m = m["channel_values"]["messages"]
mm = _validate_chat_history(messages_m)
isinstance(mm, ToolMessage)

False

In [80]:
messages_l

[HumanMessage(content='quero trabalhar com van escolar. o que eu preciso pra conseguir a autorização da prefeitura? o carro tem que ter o que?', additional_kwargs={}, response_metadata={}, id='26470c70-6781-467c-9a0e-f826b46a8de2'),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'google_search', 'arguments': '{"query": "autoriza\\u00e7\\u00e3o van escolar prefeitura rio de janeiro requisitos"}'}}, response_metadata={'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 4482, 'candidates_token_count': 13, 'total_token_count': 4597, 'cached_content_token_count': 4366, 'prompt_tokens_details': [{'modality': 1, 'token_count': 4482}], 'cache_tokens_details': [{'modality': 1, 'token_count': 4366}], 'candidates_tokens_details': [{'modality': 1, 'token_count': 13}], 'thoughts_token_count': 102}, 'finish_reason': 'STOP', 'avg_logprobs': -1.3487435854398286, 'model_name': 'gemini-2.5-flash'}, id='run--11744d20-998e-4fab-82e7-cd352e3120e6-0', too

In [79]:
messages_m

[HumanMessage(content='quem é o presidente de tuvalu? utilize a tool google_search', additional_kwargs={}, response_metadata={}, id='acdcb3ba-7816-4c32-9daa-97c3c9cc49fd'),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'google_search', 'arguments': '{"query": "presidente de tuvalu"}'}}, response_metadata={'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 3976, 'candidates_token_count': 8, 'total_token_count': 4029, 'prompt_tokens_details': [{'modality': 1, 'token_count': 3976}], 'candidates_tokens_details': [{'modality': 1, 'token_count': 8}], 'thoughts_token_count': 45, 'cached_content_token_count': 0, 'cache_tokens_details': []}, 'finish_reason': 'STOP', 'avg_logprobs': -1.2998656034469604, 'model_name': 'gemini-2.5-flash'}, id='run--754f561b-8024-421c-acff-f8e74292b765-0', tool_calls=[{'name': 'google_search', 'args': {'query': 'presidente de tuvalu'}, 'id': '96e4778a-3983-4c23-9ee8-66632ca51451', 'type': 'tool_call'}], usage_m

In [ ]:
# list = checkpointer.alist(config={"configurable": {"thread_id": user_id}})
# print(list)